# CPN generator

## Table of contents
1. [Libraries](##Libraries)
2. [Important Variables](##important-variables)
3. [Language Setting](#language-setting)
4. [Important functions](##important-functions)
5. [Data Cleaning](#data-cleaning)
6. [Word Embedding](##word-embedding)
7. [Model](#model)
8. [Codification Suggested](#codification-suggested)

## [Libraries](##Libraries)

In [1]:
# Uncomment to install all the requirement
# ! pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import spacy
import gensim
from collections import Counter 
import sklearn.cluster
from sklearn.model_selection import KFold
import random
from scipy.spatial import distance
import time

# Set of the random seed for the kmeans model
random.seed(0)

## [Important Variables](##important-variables)

In [3]:
# PATHS
# This the path to run the main example, for your data change the path to your data
 
pathTrainData = r'../data/CPN27/CPN27.csv'              # Path of the training data
pathData = r'../data/CPN27/CPN27-to-Codify.csv'         # Path of the data to codify
pathEmbedding = r'../data/SBW-vectors-300-min5.bin.gz'  # For example in Spanish
outputFile = r'../data/ACPLT-output.csv'                # Path for the results of the AC-PLT

In [4]:
# VARIABLES
numberCluster = 500 # Number of clusters for the k-means model
numberCodes = 3 # Number of code you want as suggestion.
language = "spanish" # You can change to "english"
embeddingLength = 300 # Change by the length of the embedding to be used

## [Language Setting](#language-setting)

In [5]:
# Data import
# Relative Path of the dataset, change for your dataset
data_train = pd.read_csv(pathTrainData, delimiter=",")

# Import of the data to codify
data_to_code = pd.read_csv(pathData, delimiter=",")


if language=="spanish":
    # Import of the model of the spanish billion words embeddings
    model = gensim.models.KeyedVectors.load_word2vec_format( 
    pathEmbedding, # Relative path of the vector
    binary=True # The model is in binary format
    ) 
else:
    # Import of the model of the word2vec-google-news-300 for English dataset
    model = gensim.models.KeyedVectors.load_word2vec_format( 
    pathEmbedding, # Relative path of the vector
    binary=False # The model is in binary format
    )    


In [6]:
# For the model to work correctly, it needs 3 columns (Concept, Feature, and Code in that order), the name itself is not important.
# Code is important for the training of the model.
data.head(5) # Example with used dataset

,Palabra (concepto),Descripción,Codificación
0,Compasión,sentimiento,sentimiento
1,Compasión,por lo que se le perdona la vida a alguien en ...,perdón
2,plan,necesario para cumplir objetivos,objetivos
3,plan,organizar recursos o personas,organización
4,plan,estrategia,estrategia


Installation of the Spanish pipeline. Change depending on the language you are using it.

More info in the page of spacy.

In [7]:
# Download each one depending on your language preference

if language=="spanish":
    !python -m spacy download es_core_news_sm    
else:
    !python -m spacy download en_core_news_sm


In [8]:
# Configuration of spacy
if language=="spanish":
    nlp = spacy.load('es_core_news_sm')
else:
    nlp = spacy.load('en_core_news_sm')

## [Important functions](##important-functions)

In [9]:
def clean_text(text):
    """
    Receive a string and return tokens of each word, without punctuations
    and in lowercase
    """
    # for each token in the sentence, add the token to the list if is not a punctuation
    return [t for t in nlp(text.lower()) if not t.is_punct]


def normalize(tokens):
    """
    Receive al list of tokens and return in one string without stop words
    """
    # for each token if is not a stop word add the word to the list
    words = [t.orth_ for t in tokens if not t.is_stop]

    # return the tokens in one string
    return(" ".join(words))

def lemmatize(tokens):
    """
    Receive al list of tokens and return in one string without stop words 
    and Lemmatized
    """
    # for each token if is not a stop word add the lemma of the word in the list
    lemmas = [t.lemma_ for t in tokens if not t.is_stop]

    # return the tokens in one string
    return(" ".join(lemmas))


def to_vector(texto, model):
    """ 
    Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence
    """
    tokens = texto.split() # splits the text by space and returns a list of words
    vec = np.zeros(embeddingLength) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is both in the word embedding
            vec += model[word] # adds every word embedding to the vector
    return vec / np.linalg.norm(vec) if np.linalg.norm(vec)>0 else vec # divides the vector by their normal

## [Data Cleaning](##data-cleaning)

In [10]:
start = time.time()
# Normalize of the feature
for i in range(len(data_train)):
    data_train.iloc[i,1] = normalize(clean_text(data_train.iloc[i,1])) # Change for the name of your dataset

for i in range(len(data_to_code)):
    data_to_code.iloc[i,1] = normalize(clean_text(data_to_code.iloc[i,1])) # Change for the name of your dataset
end = time.time()
print("Execution time:", end-start)

Exceution time: 23.832730293273926


## [Word Embedding](##word-embedding)

Word Embedding for the training dataset

In [11]:
# Timer
start = time.time()


# Creation of the matrix to make the clustering process
descriptions_matrix = np.zeros( # creatign an empty matrix
    (
        len(data_train), # the number of data points
        embeddingLength  # the number of components of the word embedding
    )
)

# Matrix filling 
# Change to the name of the descriptions of your dataset.
for i,description in enumerate(data_train.iloc[:,1]):
    vector = to_vector(description,model)
    descriptions_matrix[i,] = vector

# Concatenate the matrix with the data of each observation
data_matrix = np.concatenate([descriptions_matrix,data_train], axis=1)


# Remove of the 'Nan' data
data_matrix = data_matrix[~pd.isnull(data_matrix[:,:300]).any(axis=1)]


end = time.time()
print("Execution time:", end-start)

Exceution time: 0.7288105487823486


Word Embedding for the dataset to code

In [12]:
# Timer
start = time.time()


# Creation of the matrix to make the clustering process
descriptions_matrix = np.zeros( # creatign an empty matrix
    (
        len(data_to_code), # the number of data points
        300                # the number of components of the word embedding
    )
)

# Matrix filling of the dataset to codify
for i,description in enumerate(data_to_code.iloc[:,1]):
    vector = to_vector(description,model)
    descriptions_matrix[i,] = vector

# Concatenate the matrix with the data of each observation
matrix_to_code = np.concatenate([descriptions_matrix,data_to_code], axis=1)


# Remove of the 'Nan' values in the data
matrix_to_code = matrix_to_code[~pd.isnull(matrix_to_code[:,:300]).any(axis=1)]


end = time.time()
print("Execution time:", end-start)

Exceution time: 0.00703120231628418


## [Model](#model)

We create a class for the text classification, were initially you have to set the number of clusters you want to use for your model.

In [13]:
class AC_PLT:

    def __init__(self, n_clusters = 500, random_state=0):
        """
        n_clusters: number of cluster in the k-Means model
        """
        
        self.n_clusters = n_clusters # number of clusters
        self.KMeans_dict = {} # dictionary of all the humans codifications for each Cluster
        self.KMeans_categories = {} # dictionary for the most frecuent value in the centroid
        self.km = sklearn.cluster.KMeans(           # creates de k-means object
            n_clusters=self.n_clusters, 
            random_state=random_state,
            n_init='auto'
        ) 
        
        
    def most_frequent(self, List): 
        """
        Receives a list of words, and return the word most frequente of
        the list
        """
        # counter of occurence of a code in a list
        occurence_count = Counter(List) 
        
        # Return the first code with more occurence
        return occurence_count.most_common(1)[0][0] 


    def fit(self, train):
        """
        Receives the train dataset and the number of clusters to train 
        the k-means model
        """
        # Train the k-means algorithm
        self.km.fit(train[:,:300])

        # Dataframe of train dataset
        df = pd.DataFrame(
            np.concatenate([
                np.reshape(train[:,302], (-1, 1)),          # Human codification
                np.reshape(self.km.labels_, (-1, 1)),       # Number of the KMean centroid
                np.reshape(train[:,300], (-1, 1))           # Concept of the codification
                ], axis=1), 
            columns=['Human', 'KMeans', 'Concept'])

        # create a dictionary of all the humans codifications for each Cluster
        self.KMeans_dict = df.groupby(by='KMeans')['Human'].apply(list).to_dict()

        # Fill a dictionary with the most frecuent value in the centroid
        for key, val in self.KMeans_dict.items():
            self.KMeans_categories[key] = self.most_frequent(val)
        
        # Generates the prediction for the train dataset
        df['KM_Prediction'] = df['KMeans'].map(self.KMeans_categories)


    def get_distances(self, test):
        """
        Receives the test data to calculate the distances of each frase, return 
        a matrix with the distances sorted
        """
        
        # Distance matrix of each test point to each cluster center
        distance_matrix = distance.cdist(test[:,:300].astype(float), self.km.cluster_centers_, 'euclidean')
        
        # Sorting distances
        self.topk=np.argsort(distance_matrix,axis=1)
        
    
    def set_labels(self):
        """
        Create a new matrix from the clusters sorted and change the value
        from numeric to the string according the codification
        """
        # Change of the numeric value to the codification 
        self.topKS=pd.DataFrame(self.topk)

        # create a temporal array of the kmeans categories
        tempData = np.array([value for (_, value) in sorted(self.KMeans_categories.items())])
        
        # print(tempData)

        # for each cluster center
        for j in range(self.topKS.shape[1]):
            # set the codification of the numeric value in the topk list
            self.topKS.iloc[:,j]=tempData[self.topk[:,j]]


    def get_accuracies(self, test):
        """
        Receives the test matrix and return the accuracies of the 
        diferents predictions
        """
        #Creating the accuracy table to check each data point
        testLabel=np.zeros(self.topKS.shape)
        indexes_method0=pd.DataFrame(np.zeros((self.topKS.shape[0],2)), columns=['index', 'value']) 

        #For each data point
        for i in range(testLabel.shape[0]):
            #Checking if some of the cluster is able to classify it right
            boolClass=self.topKS.iloc[i,:]==test[i,302]
            if sum(boolClass)>0:
                getIndex=boolClass.idxmax()
                indexes_method0.iloc[i,0] = getIndex
                indexes_method0.iloc[i,1] = self.topKS.iloc[i,getIndex]
                #Setting the rest of the data point as 1
                testLabel[i,getIndex:]=1
            else:
                indexes_method0.iloc[i,0] = np.nan
                indexes_method0.iloc[i,1] = np.nan
        accuracies=testLabel.sum(axis=0)/testLabel.shape[0]

        return accuracies

    
    def transform(self, test):
        """
        Receives two numpy bi-dimentionals arrays and returns the accuracy of the model
        """
        self.get_distances(test)
        self.set_labels()
        return self.get_accuracies(test)
    
    def suggestions(self, test, n_codes):
        self.get_distances(test)
        self.set_labels()
        return pd.DataFrame(
            np.concatenate([
                np.reshape(test[:, 300], (-1, 1)), 
                np.reshape(test[:, 301], (-1, 1)), 
                self.topKS.iloc[:, :n_codes]],
                axis=1
                ), 
            columns=['Concept', 'Description']+['top-{} suggestion'.format(i+1) for i in range(n_codes)]
            )
        
    def get_inertia(self): 
        """
        Return the inertia of the current model
        """
        return self.km.inertia_

## [Codification Suggested](#codification-suggested)

In [14]:
# Instance of the model
proc = AC_PLT(n_clusters=numberCluster)

# Training process
start = time.time()
proc.fit(data_matrix)
end = time.time()
print("Train process time:", end-start)

# Creation of Dataframe with the suggested codes
start = time.time()
df = proc.suggestions(matrix_to_code, n_codes=numberCodes)
end = time.time()
print("Clasification process time:", end-start)

# Saving the dataframe into a csv file
df.to_csv(outputFile, index=False)

Train process time: 1.742814064025879
Clasification process time: 0.09805965423583984
